# Multi-Agents Orchestration Notebook

Ce notebook illustre un pipeline complet pour orchestrer la collaboration de plusieurs agents (Coder, Reviewer, Admin) autour d’un **notebook cible** à finaliser.

---

### Étapes principales

1. **Sélection du notebook à finaliser** (via un prompt ou dropdown `ipywidgets`).
2. **Objectif facultatif** : l’utilisateur peut préciser un objectif global qui sera injecté en contexte si présent.
3. **Chargement d’un état partagé** (`NotebookState`) permettant :
   - de conserver un cache d’exécution (pour éviter de relancer Papermill après chaque modification si le notebook n’a pas changé)  
   - de gérer les modifications successives du `Coder`  
4. **Définition des plugins** (Coder, Reviewer, Admin) :
   - Coder : `update_cell`  
   - Reviewer : `get_notebook_status`  
   - Admin : `submit_for_approval`
5. **Création d’un `AgentGroupChat`** multi-agents sous Semantic Kernel, plus un système de **function calling** centralisé pour manipuler le notebook.
6. **Boucle itérative** gérée par des stratégies de sélection et de terminaison (Semantic Kernel) pour arrêter la conversation une fois le notebook approuvé.

---


In [117]:
# Bloc 1 : Installation et imports (à exécuter une seule fois)
%pip install --quiet papermill nbformat ipywidgets semantic-kernel

#!/usr/bin/env python
# coding: utf-8

import os
import logging
import nbformat
import papermill as pm
import hashlib
from datetime import datetime

# IPython.display pour des affichages enrichis
from IPython.display import display, HTML, Markdown

# -- Imports Semantic Kernel
import asyncio
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.contents import ChatHistory, AuthorRole, ChatMessageContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import (
    KernelFunctionTerminationStrategy,
    KernelFunctionSelectionStrategy
)
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy



print("Packages installés et imports effectués.")


Note: you may need to restart the kernel to use updated packages.
Packages installés et imports effectués.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Configuration du Logger

Ici, nous activons un logger plus verbeux, avec couleurs, pour afficher toutes les actions (modifications du notebook, appels de fonctions, messages entre agents, etc.).


In [118]:
# --- Cellule : Configuration Logger ---

import logging
from datetime import datetime

class ColorFormatter(logging.Formatter):
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record):
        # Couleurs selon le niveau
        color = self.colors.get(record.levelname, '')
        message = super().format(record)
        return f"{color}{message}{self.reset}" if color else message

logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)
logger.propagate = False  # Évite de propager vers le root logger

if not any(isinstance(h, logging.StreamHandler) for h in logger.handlers):
    handler = logging.StreamHandler()
    handler.setFormatter(ColorFormatter(
        '%(asctime)s [%(levelname)s] %(name)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    ))
    logger.addHandler(handler)

# Désactiver logs papermill, openai, etc.
logging.getLogger("papermill").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("semantic_kernel").setLevel(logging.WARNING)


# Dans le bloc d'initialisation des logs - Modifier la classe ColorFormatter
def format(self, record):    
    notebook_info = getattr(record, 'notebook_snapshot', '')
    agent_action = getattr(record, 'agent_action', '')
    msg = super().format(record)
    if notebook_info:
        msg += f"\n💻 NOTEBOOK CONTEXT: {notebook_info[:200]}..."
    if agent_action:
        msg += f"\n🤖 AGENT ACTION: {agent_action}"
    return msg




logger.info("Logger correctement configuré.")


2025-02-10 06:21:53 [INFO] Orchestration - Logger correctement configuré.
2025-02-10 06:21:53,131 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | Logger correctement configuré.
2025-02-10 06:21:53,131 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | Logger correctement configuré.


## Sélection du notebook à finaliser et Objectif facultatif

On propose un sélecteur `ipywidgets` si disponible, sinon on tombera sur un `input()` classique.  
On saisit aussi un objectif (string) qui reste **optionnel**.  


In [119]:
# Version simplifiée
notebook_files = [f for f in os.listdir('.') if f.endswith('.ipynb')]
selected_notebook = 'Notebook-Template.ipynb'  # Valeur par défaut

try:
    import ipywidgets as widgets
    dropdown = widgets.Dropdown(options=notebook_files, description='Notebook:')
    display(dropdown)
    selected_notebook = dropdown.value
except:
    print("Fichiers disponibles:", notebook_files)
    selected_notebook = input("Nom du notebook cible: ")


user_objective = input("Entrez un objectif (facultatif), ou laissez vide : ").strip()

logger.info(f"Notebook cible: {selected_notebook}, Objective facultatif: {user_objective}")


Dropdown(description='Notebook cible:', index=8, layout=Layout(width='50%'), options=('01-SemanticKernel-Intro…

2025-02-10 06:23:33 [INFO] Orchestration - Notebook cible: None, Objective facultatif: 
2025-02-10 06:23:33,506 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | Notebook cible: None, Objective facultatif: 
2025-02-10 06:23:33,506 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | Notebook cible: None, Objective facultatif: 


## Partie 1 : Exécution paramétrée et mise à jour du Notebook

1. Nous chargeons un notebook template (Notebook-Template.ipynb) et y injectons l'instruction de tâche via Papermill.
2. Ensuite, nous mettons à jour une cellule ciblée (repérant le marqueur unique TASK_INSTRUCTION_PLACEHOLDER) via nbformat, puis nous réexécutons le notebook pour obtenir la version finale.

In [120]:
# Exécution du notebook template avec Papermill
source_notebook = "Notebook-Template.ipynb" 
output_notebook = "Notebook-Updated.ipynb"
final_notebook = "Notebook-Final.ipynb"

task_instruction = "Créer un notebook Python qui requête DBpedia via SPARQL et affiche un graphique Plotly."

logger.info("🚀 Début de l'exécution Papermill avec paramètre: %s", task_instruction)
pm.execute_notebook(
    source_notebook,
    output_notebook,
    parameters=dict(TASK_INSTRUCTION=task_instruction)
)
logger.info("✅ Papermill a généré: %s", output_notebook)
print(f"Notebook exécuté sous '{output_notebook}'.")

# Fonction de mise à jour enrichie avec logging
def update_notebook_cell(notebook_path: str, unique_marker: str, new_content: str):
    try:
        logger.info(f"🔍 Recherche du marqueur '{unique_marker}' dans {notebook_path}")
        nb = nbformat.read(notebook_path, as_version=4)
        logger.info(f"📄 Nombre de cellules initiales : {len(nb.cells)}")
        
        found = False
        for idx, cell in enumerate(nb.cells):
            source = cell.get("source", "")
            if unique_marker in source:
                logger.info(f"🎯 Cellule trouvée à l'index {idx} - Type: {cell.cell_type}")
                logger.info("👉 Avant mise à jour : %s", source[:100].replace("\n", " "))
                cell["source"] = new_content
                nbformat.write(nb, notebook_path)
                logger.info("✅ Cellule %d mise à jour avec succès", idx)
                logger.info("👉 Après mise à jour : %s", cell["source"][:100].replace("\n", " ") + ("..." if len(cell["source"]) > 100 else ""))
                found = True
                break
        
        if not found:
            error_msg = f"❌ Marqueur '{unique_marker}' non trouvé parmi {len(nb.cells)} cellules"
            logger.error(error_msg)
            raise ValueError(error_msg)
    except Exception as e:
        logger.exception(f"💥 Échec critique lors de la mise à jour du notebook: {str(e)}")
        raise



# Appel avec log du résultat
update_notebook_cell(output_notebook, "TASK_INSTRUCTION_PLACEHOLDER", 
                     f"TASK_INSTRUCTION_PLACEHOLDER\nTask: {task_instruction}\nUpdated by AI Agent")

logger.info("🔄 Réexécution du notebook mis à jour")
pm.execute_notebook(output_notebook, final_notebook)
logger.info("✅ Notebook final généré: Notebook-Final.ipynb")



2025-02-10 06:23:33 [INFO] Orchestration - 🚀 Début de l'exécution Papermill avec paramètre: Créer un notebook Python qui requête DBpedia via SPARQL et affiche un graphique Plotly.
2025-02-10 06:23:33,540 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 🚀 Début de l'exécution Papermill avec paramètre: Créer un notebook Python qui requête DBpedia via SPARQL et affiche un graphique Plotly.
2025-02-10 06:23:33,540 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 🚀 Début de l'exécution Papermill avec paramètre: Créer un notebook Python qui requête DBpedia via SPARQL et affiche un graphique Plotly.
Passed unknown parameter: TASK_INSTRUCTION
Input notebook does not contain a cell with tag 'parameters'
Executing: 100%|██████████| 12/12 [00:04<00:00,  2.96cell/s]
2025-02-10 06:23:37 [INFO] Orchestration - ✅ Papermill a généré: Notebook-Updated.ipynb
2025-02-10 06:23:37,607 | INFO | 

Notebook exécuté sous 'Notebook-Updated.ipynb'.


Executing: 100%|██████████| 12/12 [00:01<00:00,  6.96cell/s]
2025-02-10 06:23:39 [INFO] Orchestration - ✅ Notebook final généré: Notebook-Final.ipynb
2025-02-10 06:23:39,381 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | ✅ Notebook final généré: Notebook-Final.ipynb
2025-02-10 06:23:39,381 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | ✅ Notebook final généré: Notebook-Final.ipynb


## Partie 1bis : Visualisation Initiale du Notebook

Avant de lancer la conversation entre agents, affichons une vue structurée du notebook template pour donner le contexte initial :

In [121]:

# Nouvelle cellule après l'exécution de Papermill (Bloc 2)

from IPython.display import display, HTML, Markdown

# Nouvelle version enrichie de display_notebook
def display_notebook(notebook_path: str, max_cells: int = 5):
    try:
        nb = nbformat.read(notebook_path, as_version=4)
        display(HTML(f"<h3 style='color:#26608e'>📓 Contenu initial de {notebook_path}</h3>"))
        
        for i, cell in enumerate(nb.cells[:max_cells]):
            cell_type = cell.cell_type.capitalize()
            header = f"### Cellule {i} ({cell_type})"
            content = cell.source.replace("\n", "<br>")[:500] + ("..." if len(cell.source) > 500 else "")
            
            # Ajout d'une couleur de bordure selon le type
            border_color = "#e0e0e0" if cell_type == "Code" else "#f0f0f0"
            
            display(HTML(f"""
                <div style='border-left:4px solid {border_color}; padding:10px; margin:15px; border-radius:4px'>
                    <div style='font-family: Monaco, monospace; font-size:0.9em; color:#404040'>{content}</div>
                </div>
            """))
            logger.info(f"📄 Cell {i} preview: {cell.source[:500].strip()}...")
            
    except Exception as e:
        logger.error(f"🚨 Erreur d'affichage du notebook : {str(e)}")
        raise

# Afficher le template initial avant l'exécution
display_notebook(source_notebook)



2025-02-10 06:23:39 [INFO] Orchestration - 📄 Cell 0 preview: # Notebook de travail

Ce notebook est généré de façon incrémentale pour accomplir la tâche décrite ci-dessous....
2025-02-10 06:23:39,431 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 0 preview: # Notebook de travail

Ce notebook est généré de façon incrémentale pour accomplir la tâche décrite ci-dessous....
2025-02-10 06:23:39,431 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 0 preview: # Notebook de travail

Ce notebook est généré de façon incrémentale pour accomplir la tâche décrite ci-dessous....


2025-02-10 06:23:39 [INFO] Orchestration - 📄 Cell 1 preview: ## 0. Objectif du Notebook

Dans cette section, nous décrivons l'objectif du notebook, sa fonction, les outils à utiliser et les buts à atteindre.

### Tâche originale

Voilà la tâche telle qu'elle a été initialement formulée :

{{TASK_DESCRIPTION}}

TASK_INSTRUCTION_PLACEHOLDER

### Interprétation et sous-objectifs

Décrire ici l'interprétation de la tâche et les étapes prévues pour la réaliser....
2025-02-10 06:23:39,436 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 1 preview: ## 0. Objectif du Notebook

Dans cette section, nous décrivons l'objectif du notebook, sa fonction, les outils à utiliser et les buts à atteindre.

### Tâche originale

Voilà la tâche telle qu'elle a été initialement formulée :

{{TASK_DESCRIPTION}}

TASK_INSTRUCTION_PLACEHOLDER

### Interprétation et sous-objectifs

Décrire ici l'interprétation de la tâche et les étapes prévues pour l

2025-02-10 06:23:39 [INFO] Orchestration - 📄 Cell 2 preview: ## 1. Préparation de l'environnement

Nous allons installer et importer les composants nécessaires....
2025-02-10 06:23:39,440 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 2 preview: ## 1. Préparation de l'environnement

Nous allons installer et importer les composants nécessaires....
2025-02-10 06:23:39,440 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 2 preview: ## 1. Préparation de l'environnement

Nous allons installer et importer les composants nécessaires....


2025-02-10 06:23:39 [INFO] Orchestration - 📄 Cell 3 preview: # Cellule 1...
2025-02-10 06:23:39,445 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 3 preview: # Cellule 1...
2025-02-10 06:23:39,445 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 3 preview: # Cellule 1...


2025-02-10 06:23:39 [INFO] Orchestration - 📄 Cell 4 preview: ## 2. Initialisation

Dans cette partie, nous créons les objets nécessaires pour réaliser la tâche....
2025-02-10 06:23:39,450 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 4 preview: ## 2. Initialisation

Dans cette partie, nous créons les objets nécessaires pour réaliser la tâche....
2025-02-10 06:23:39,450 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📄 Cell 4 preview: ## 2. Initialisation

Dans cette partie, nous créons les objets nécessaires pour réaliser la tâche....


## Partie 2 : Intégration d'un Plugin de Contrôle du Notebook

Nous définissons un plugin (NotebookControlPlugin) qui expose des fonctions pour :
- Obtenir le statut du notebook (par exemple, le nombre de cellules, les erreurs éventuelles) via `get_notebook_status`.
- Soumettre le notebook pour approbation via `submit_for_approval`.

Ces fonctions permettront aux agents d'interroger l'état du notebook et de déclencher des actions.

La classe NotebookState centralisera l'ensemble des interactions réalisées avec un notebook jupyter

In [122]:
class NotebookState:
    """État centralisé du notebook avec suivi des modifications et cache"""
    
    def __init__(self, notebook_path: str):
        self.notebook_path = notebook_path
        self._nb = None
        self.modification_history = []
        self.last_execution = None
        self.reload()
        
    def reload(self):
        """Recharge le notebook depuis le disque et met à jour le cache"""
        try:
            self._nb = nbformat.read(self.notebook_path, as_version=4)
            logger.debug(f"🔄 Notebook rechargé ({len(self._nb.cells)} cellules)")
        except Exception as e:
            logger.error(f"⛔ Erreur de rechargement : {str(e)}")
            raise
            
    @property
    def cells(self) -> list:
        """Accès direct aux cellules"""
        return self._nb.cells
    
    @property
    def metadata(self) -> dict:
        """Métadonnées du notebook"""
        return self._nb.metadata
    
    @property
    def cell_count(self) -> int:
        return len(self.cells)
    
    def find_cell_index(self, marker: str) -> int:
        """Trouve l'index d'une cellule par son contenu (marker)"""
        for idx, cell in enumerate(self.cells):
            if marker in cell.source:
                return idx
        return -1
    
    def update_cell(self, index: int, new_content: str, agent: str = "System") -> None:
        """Met à jour une cellule de manière atomique"""
        if index < 0 or index >= self.cell_count:
            raise ValueError(f"Index {index} invalide")
        
        old_content = self.cells[index].source
        self.cells[index].source = new_content
        
        # Historique des modifications
        self.modification_history.append({
            "timestamp": datetime.now().isoformat(),
            "agent": agent,
            "index": index,
            "old": old_content[:500],
            "new": new_content[:500]
        })
        
        self._save()
        logger.info(f"✏️ Cellule {index} modifiée par {agent}", extra={
            "diff": f"{old_content[:100]}... → {new_content[:100]}...",
            "index": index
        })
        
    def _save(self):
        """Sauvegarde atomique du notebook"""
        try:
            with open(self.notebook_path, 'w', encoding='utf-8') as f:
                nbformat.write(self._nb, f)
            logger.debug(f"💾 Notebook sauvegardé ({self.cell_count} cellules)")
        except Exception as e:
            logger.error(f"⛔ Échec sauvegarde : {str(e)}")
            raise
            
    def execute(self, output_path: str = None):
        """Exécute le notebook via Papermill et met à jour l'état"""
        output_path = output_path or self.notebook_path.replace(".ipynb", "-EXECUTED.ipynb")
        
        try:
            pm.execute_notebook(
                self.notebook_path,
                output_path,
                kernel_name="python3",
                progress_bar=False
            )
            self.last_execution = datetime.now()
            logger.info(f"✅ Notebook exécuté sous {output_path}")
            return output_path
        except Exception as e:
            logger.error(f"⛔ Échec d'exécution : {str(e)}")
            raise


### Définition des plugins

In [123]:



# ---------------------------------
# On crée l'état partagé
# ---------------------------------
shared_state = NotebookState(final_notebook)

logger.info(f"NotebookState initialisé pour: {final_notebook}")




# Nouvelle fonction à ajouter
def display_cell_diff(notebook_path: str, cell_index: int, before: str, after: str):
    from IPython.display import display, HTML
    diff_html = f"""
    <div style='border:1px solid #ddd; padding:10px; margin:10px; background:#f8f8f8'>
        <strong>Cellule {cell_index} modifiée</strong>
        <div style='display:flex; justify-content:space-between; margin-top:5px'>
            <div style='width:48%; color:#d00'><pre>{html.escape(before)}</pre></div>
            <div style='width:48%; color:#0a0'><pre>{html.escape(after)}</pre></div>
        </div>
    </div>
    """
    display(HTML(diff_html))




class NotebookPluginBase:
    """Classe de base pour tous les plugins notebook"""
    
    def __init__(self, state: NotebookState):
        self.state = state
        self.logger = logging.getLogger(f"Plugin.{self.__class__.__name__}")
        
    def _log_action(self, action: str, **kwargs):
        """Journalisation standardisée des actions"""
        self.logger.info(
            f"🔧 {action}",
            extra={
                "notebook": self.state.notebook_path,
                "cells": self.state.cell_count,
                **kwargs
            }
        )



class CoderPlugin(NotebookPluginBase):
    
    @kernel_function(name="update_marked_cell", description="Met à jour une cellule identifiée par un marqueur")
    def update_marked_cell(self, marker: str, new_content: str, agent: str = "Coder") -> str:
        try:
            index = self.state.find_cell_index(marker)
            if index == -1:
                raise ValueError(f"Marqueur '{marker}' introuvable")
                
            self.state.update_cell(index, new_content, agent)
            self._log_action("Cellule modifiée", index=index, marker=marker)
            return f"✅ Cellule {index} mise à jour"
            
        except Exception as e:
            self.logger.error(f"Échec modification cellule : {str(e)}")
            raise



class ReviewerPlugin(NotebookPluginBase):
    
    @kernel_function(name="validate_notebook", description="Exécute et valide le notebook")
    def validate_notebook(self) -> str:
        try:
            # Exécute et recharge automatiquement
            executed_path = self.state.execute()
            self.state.reload()
            
            # Analyse des erreurs
            error_count = sum(
                1 for cell in self.state.cells
                if cell.cell_type == "code" and any(
                    output.output_type == "error" 
                    for output in cell.get("outputs", [])
                )
            )
            
            status = "VALIDE" if error_count == 0 else f"INVALIDE ({error_count} erreurs)"
            return f"📊 Rapport de validation : {status}"
            
        except Exception as e:
            self.logger.error(f"Échec validation : {str(e)}")
            return f"❌ Échec validation : {str(e)}"



class AdminPlugin(NotebookControlPluginBase):
    """Plugin pour les opérations d'approbation finale"""
    def __init__(self, state: NotebookState):
        super().__init__(state)
        self.logger = logging.getLogger("Orchestration.AdminPlugin")

    @kernel_function(name="final_approval", description="Soumet pour approbation finale")
    def final_approval(self) -> str:
        self.logger.info("📝 Soumission du notebook '%s' pour approbation.", self.state.notebook_path)
        return f"✅ Notebook '{self.state.notebook_path}' soumis et approuvé."


   



2025-02-10 06:23:39 [DEBUG] Orchestration - 🔄 Notebook rechargé (12 cellules)
2025-02-10 06:23:39,662 | DEBUG | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 🔄 Notebook rechargé (12 cellules)
2025-02-10 06:23:39,662 | DEBUG | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 🔄 Notebook rechargé (12 cellules)
2025-02-10 06:23:39 [INFO] Orchestration - NotebookState initialisé pour: Notebook-Final.ipynb
2025-02-10 06:23:39,667 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | NotebookState initialisé pour: Notebook-Final.ipynb
2025-02-10 06:23:39,667 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | NotebookState initialisé pour: Notebook-Final.ipynb


## Partie 3 : Création des Agents (Coder, Reviewer, Admin) avec Plugins

Chaque agent aura son propre kernel auquel sera ajouté un plugin spécifique lui permettant d'accéder aux fonctions de contrôle du notebook.
- Le **Coder Agent** utilisera NotebookControlPluginCoder pour éditer les cellules.
- Le **Reviewer Agent** utilisera NotebookControlPluginReviewer pour obtenir le statut du notebook.
- L’**Admin Agent** utilisera NotebookControlPluginAdmin pour soumettre le notebook pour approbation.

In [124]:
def create_kernel_for_agent(service_id: str, plugin_instance) -> Kernel:
    k = Kernel()
    # Ajout du service OpenAIChatCompletion pour cet agent
    k.add_service(OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-4o-mini"))
    # Ajout du plugin spécifique, si fourni
    if plugin_instance is not None:
        k.add_plugin(plugin_instance, plugin_name="nbcontrol")
    return k

def notebook_context_for_agent(notebook_path: str, focus_cell: int = -1) -> str:
    try:
        nb = nbformat.read(notebook_path, as_version=4)
        ctx = [
            f"### Notebook {notebook_path} ({len(nb.cells)} cellules)"            
        ]
        if focus_cell >= 0 and focus_cell < len(nb.cells):
            cell = nb.cells[focus_cell]
            ctx.append(f"Cellule {focus_cell} ({cell.cell_type}): {cell.source[:200]}...")
        return "\n".join(ctx)
    except Exception as e:
        return f"⚠️ Erreur de lecture : {str(e)}"





# Instructions et noms pour chaque agent
CODER_AGENT_NAME = "Coder_Agent"
# CODER_AGENT_INSTRUCTIONS = "Vous êtes le Coder. Vous devez mettre à jour les cellules de code selon l'instruction fournie."

# CODER_AGENT_INSTRUCTIONS = f"""
# Vous êtes le Coder. Vous devez mettre à jour les cellules de code selon l'instruction fournie.
# Contexte actuel : {notebook_context_for_agent(shared_state.notebook_path)}
# ...
# """

# --- Nouveau : injection du notebook en contexte pour le coder ---
def build_coder_instructions():
    notebook_summary = notebook_context_for_agent(shared_state.notebook_path)
    base_instructions = f"""
Vous êtes le Coder. Vous devez mettre à jour les cellules du notebook en fonction des besoins exprimés.
Voici un résumé du notebook pour vous aider à comprendre sa structure actuelle :

{notebook_summary}

Pour modifier une cellule, vous pouvez appeler la fonction `update_cell` (plugin nbcontrol) avec un marqueur ou un index.
"""
    # On ajoute l'objectif s'il existe
    if user_objective:
        base_instructions += f"\nObjectif global à garder en tête : {user_objective}\n"

    return base_instructions.strip()



CODER_AGENT_INSTRUCTIONS = build_coder_instructions()




REVIEWER_AGENT_NAME = "Reviewer_Agent"
REVIEWER_AGENT_INSTRUCTIONS = "Vous êtes le Reviewer. Exécutez le notebook complet, vérifiez son statut via get_notebook_status et fournissez des retours."

ADMIN_AGENT_NAME = "Admin_Agent"
ADMIN_AGENT_INSTRUCTIONS = "Vous êtes l'Admin. Vérifiez que le notebook est correct et soumettez-le pour approbation via submit_for_approval."

# Création des plugins spécifiques pour chaque agent (en partageant l'état)
coder_plugin = CoderPlugin(shared_state)
reviewer_plugin = ReviewerPlugin(shared_state)
admin_plugin = AdminPlugin(shared_state)

# Création des kernels dédiés pour chaque agent
coder_kernel = create_kernel_for_agent(CODER_AGENT_NAME, coder_plugin)
reviewer_kernel = create_kernel_for_agent(REVIEWER_AGENT_NAME, reviewer_plugin)
admin_kernel = create_kernel_for_agent(ADMIN_AGENT_NAME, admin_plugin)

# Création des agents
coder_agent = ChatCompletionAgent(
    service_id=CODER_AGENT_NAME,
    kernel=coder_kernel,
    name=CODER_AGENT_NAME,
    instructions=CODER_AGENT_INSTRUCTIONS
)

reviewer_agent = ChatCompletionAgent(
    service_id=REVIEWER_AGENT_NAME,
    kernel=reviewer_kernel,
    name=REVIEWER_AGENT_NAME,
    instructions=REVIEWER_AGENT_INSTRUCTIONS
)

admin_agent = ChatCompletionAgent(
    service_id=ADMIN_AGENT_NAME,
    kernel=admin_kernel,
    name=ADMIN_AGENT_NAME,
    instructions=ADMIN_AGENT_INSTRUCTIONS
)

# Création d'un group chat avec les trois agents
group_chat = AgentGroupChat(agents=[coder_agent, reviewer_agent, admin_agent])
print("Group chat multi-agent créé avec plugins spécifiques (état partagé).")


Group chat multi-agent créé avec plugins spécifiques (état partagé).


## Partie 4 : Workflow Itératif Complet

Dans cette section, nous mettons en place un workflow itératif où :
- L'utilisateur envoie une demande de mise à jour.
- Le **Coder Agent** propose des modifications via sa fonction d'édition.
- Le **Reviewer Agent** exécute le notebook et vérifie son statut via `get_notebook_status`.
- L’**Admin Agent** intervient pour soumettre le notebook pour approbation via `submit_for_approval`.

Le passage de main (sélection du prochain agent) et la terminaison de la conversation sont gérés par des stratégies basées sur des fonctions kernel (créées à partir de prompts).

In [125]:
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy

def selection_result_parser(result):
    next_agent = selection_result_parser_internal(result)
    logger.info(f"Choix de l'agent suivant: {next_agent}", 
               extra={'selection_reason': result.value[:100]})
    return next_agent

# Pour extraire le contenu texte d'un résultat éventuel (si c'est un ChatMessageContent), on définit des parsers dédiés
def selection_result_parser_internal(result):
    response = str(result).strip()
    logger.debug(f"🔄 Réponse brute de sélection: {response}", 
               extra={'agent_action': 'SelectionDebug'})
    
    # Nettoyage de la réponse
    agent_map = {
        "coder": "Coder_Agent",
        "review": "Reviewer_Agent",
        "admin": "Admin_Agent"
    }
    
    for key, agent in agent_map.items():
        if key in response.lower():
            logger.debug(f"✅ Agent sélectionné via mapping: {agent}")
            return agent
    
    # Fallback logique
    if "review" in response.lower() or "error" in response.lower():
        return "Reviewer_Agent"
    return "Coder_Agent"


def termination_result_parser(result):
    value = result.value
    if value is None:
        return False
    if isinstance(value, list):
        first = value[0]
        text = first.content if hasattr(first, "content") else str(first)
    elif hasattr(value, "content"):
        text = value.content
    else:
        text = str(value)
    return "approved" in text.lower()


# Définition d'une fonction de terminaison via prompt
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
Examine the conversation history and determine if the notebook is approved.
If the notebook is approved, respond with a single word: "approved".
Otherwise, respond with "continue".
History:
{{$history}}
"""
)

# Dans la définition de selection_function (Cell 5), REMPLACER le prompt par :
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""Analysez l'historique et le contexte notebook.
Agents disponibles: 
- Coder_Agent (Éditeur de code)
- Reviewer_Agent (Validateur)
- Admin_Agent (Approbation finale)

Règles :
1. Après une erreur → Reviewer_Agent
2. Après validation OK → Admin_Agent
3. Sinon → Coder_Agent

Historique :
{{$history}}

Le prochain agent doit être :"""
)



# Création d'une instance de kernel pour les stratégies (pour la sélection et la terminaison)
def create_kernel_for_strategy(service_id: str) -> Kernel:
    k = Kernel()
    k.add_service(OpenAIChatCompletion(service_id=service_id))
    return k

termination_strategy = KernelFunctionTerminationStrategy(
    agents=[admin_agent],
    function=termination_function,
    kernel=create_kernel_for_strategy("termination"),
    result_parser=termination_result_parser,  # Utilisation de la fonction de parsing corrigée
    history_variable_name="history",
    maximum_iterations=10
)


selection_strategy = KernelFunctionSelectionStrategy(
    function=selection_function,
    kernel=create_kernel_for_strategy("selection"),
    result_parser=selection_result_parser,  # Utilisation de la fonction de parsing corrigée
    agent_variable_name="_agent_",
    history_variable_name="history"
)


# Création d'un nouveau group chat itératif intégrant ces stratégies
iterative_group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    termination_strategy=termination_strategy,
    selection_strategy=selection_strategy
)




## Boucle principale et exécution



In [126]:
async def run_iterative_workflow():
    chat_history = ChatHistory()
    # Injection du contexte initial avec un extrait du template
    nb_content = "\n".join([c.source[:100] for c in nbformat.read(source_notebook, as_version=4).cells[:3]])
    initial_context = f"""
## CONTEXTE INITIAL ##
Notebook template: {source_notebook}
Extrait de cellules clés:
{nb_content}
"""
    chat_history.add_system_message(initial_context)
    logger.info("📂 Contexte initial injecté : %.80s...", initial_context.replace("\n", " "))

    # Message utilisateur initial
    user_request = "Veuillez implémenter une requête SPARQL sur DBpedia avec visualisation Plotly."
    chat_history.add_user_message(user_request)
    print(f"# User: '{user_request}'")

    iteration = 0
    async for msg in iterative_group_chat.invoke():
        
        # Ajouter avant le logger existant :
        if hasattr(msg, 'function_call') and msg.function_call is not None:
            fn_name = msg.function_call.name
            fn_args = msg.function_call.arguments
            logger.info(
                f"🔧 [Function Calling] {msg.role.name} appelle la fonction '{fn_name}' avec arguments={fn_args}",
                extra={"agent_action": f"Call:{fn_name}"}
            )
        
        if msg.role.name == "ASSISTANT":
            current_cells = nbformat.read(shared_state.notebook_path, as_version=4).cells
            logger.info(
                "🧠 Contexte agent (%s): %d cellules",
                msg.role.name,
                len(current_cells)
            )
        
        
        iteration += 1
        # On stocke le numéro de l'itération dans les métadonnées du message (si ce n'est pas déjà fait)
        msg.metadata["iteration"] = iteration
        logger.info("[Tour %d] %s: %.120s...", iteration, msg.role.name, msg.content.replace("\n", " "))
        
        # Toutes les 2 itérations, on affiche l'état actuel du notebook
        if iteration % 2 == 0:
            current_state = nbformat.read(shared_state.notebook_path, as_version=4)
            logger.info("📌 État du notebook '%s': %d cellules", shared_state.notebook_path, len(current_state.cells))
    
    # Affichage du rapport final
    final_report = f"""
=== RAPPORT FINAL ===
🗂 Notebook final : {final_notebook}
🔄 Cellules modifiées : {sum(1 for c in nbformat.read(final_notebook, as_version=4).cells if "Updated by AI" in c.source)}
✅ Dernière exécution : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
📊 Statut : {"APPROUVÉ" if iterative_group_chat.is_complete else "EN ÉCHEC"}
"""
    from IPython.display import Markdown, display
    display(Markdown(final_report))
    logger.info(final_report.replace("\n", " "))
    

await run_iterative_workflow()


2025-02-10 06:23:41 [INFO] Orchestration - 📂 Contexte initial injecté :  ## CONTEXTE INITIAL ## Notebook template: Notebook-Template.ipynb Extrait de ce...
2025-02-10 06:23:41,146 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📂 Contexte initial injecté :  ## CONTEXTE INITIAL ## Notebook template: Notebook-Template.ipynb Extrait de ce...
2025-02-10 06:23:41,146 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 📂 Contexte initial injecté :  ## CONTEXTE INITIAL ## Notebook template: Notebook-Template.ipynb Extrait de ce...


# User: 'Veuillez implémenter une requête SPARQL sur DBpedia avec visualisation Plotly.'


2025-02-10 06:23:42 [DEBUG] Orchestration - 🔄 Réponse brute de sélection: Étant donné l'historique vide et l'absence d'erreurs ou de validations précédentes, le prochain agent appelé doit être le **Coder_Agent** pour commencer ou continuer le processus de développement.
2025-02-10 06:23:42,396 | DEBUG | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 🔄 Réponse brute de sélection: Étant donné l'historique vide et l'absence d'erreurs ou de validations précédentes, le prochain agent appelé doit être le **Coder_Agent** pour commencer ou continuer le processus de développement.
2025-02-10 06:23:42,396 | DEBUG | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} | 🔄 Réponse brute de sélection: Étant donné l'historique vide et l'absence d'erreurs ou de validations précédentes, le prochain agent appelé doit être le **Coder_Agent** pour commencer ou continuer le processus de développement.
2025-02-10 06


=== RAPPORT FINAL ===
🗂 Notebook final : Notebook-Final.ipynb
🔄 Cellules modifiées : 1
✅ Dernière exécution : 2025-02-10 06:24:23
📊 Statut : EN ÉCHEC


2025-02-10 06:24:23 [INFO] Orchestration -  === RAPPORT FINAL === 🗂 Notebook final : Notebook-Final.ipynb 🔄 Cellules modifiées : 1 ✅ Dernière exécution : 2025-02-10 06:24:23 📊 Statut : EN ÉCHEC 
2025-02-10 06:24:23,469 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} |  === RAPPORT FINAL === 🗂 Notebook final : Notebook-Final.ipynb 🔄 Cellules modifiées : 1 ✅ Dernière exécution : 2025-02-10 06:24:23 📊 Statut : EN ÉCHEC 
2025-02-10 06:24:23,469 | INFO | Orchestration | Notebook:{'path': 'Notebook-Final.ipynb', 'active_cell': 0, 'cells_count': 12} |  === RAPPORT FINAL === 🗂 Notebook final : Notebook-Final.ipynb 🔄 Cellules modifiées : 1 ✅ Dernière exécution : 2025-02-10 06:24:23 📊 Statut : EN ÉCHEC 


## Conclusion et Perspectives

Ce notebook intègre désormais un workflow complet et itératif qui combine :
1. L'exécution paramétrée d'un notebook template via Papermill.
2. La mise à jour dynamique d'une cellule ciblée avec nbformat.
3. Une orchestration collaborative multi‑agents (Coder, Reviewer, Admin) avec Semantic Kernel, chacun disposant d'un kernel et de plugins spécifiques pour contrôler l'état du notebook.
4. Des stratégies de sélection et de terminaison (définies via des fonctions kernel) permettant de passer la main de manière itérative jusqu'à ce que le notebook soit approuvé.

Les prochaines étapes pourraient inclure :
- L'enrichissement des plugins pour gérer d'autres aspects (logs, rollback, etc.).
- L'ajustement des stratégies de passage de main pour une sélection plus fine de l'agent suivant.
- La gestion avancée de l'historique de la conversation pour une meilleure réduction des messages.

Bonne exploration !